In [1]:
import pandas as pd 
import numpy as np
import pickle
import os

In [2]:
base_loc = "/home/naradaw/dev/Charles_Schwab/data/w_features/v2/2024_11_19/2024_11_19_09_21"

In [3]:
data_sqs_path = f"{base_loc}/dataset_sqs.pkl"

In [4]:
baseline_df = pd.read_csv(f"{base_loc}/baseline_dataset_raw.csv")
data_df = pd.read_csv(f"{base_loc}/dataset_raw.csv")
processed_df = pd.read_csv(f"{base_loc}/dataset_processed.csv")



In [5]:
baseline_df.tail()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,baseline_target_,baseline_returns_
1730,SPY,2024-10-15,584.59,584.90,578.5450,579.78,54203636.0,450436.0,581.362965,596.90,0.029528
1731,SPY,2024-10-16,579.78,582.83,578.9600,582.30,30725436.0,327311.0,581.263073,597.19,0.025571
1732,SPY,2024-10-17,585.91,586.12,582.1600,582.35,34393714.0,350466.0,583.466236,593.35,0.018889
1733,SPY,2024-10-18,584.07,585.39,582.5800,584.59,37416801.0,312698.0,584.217609,585.75,0.001984
1734,SPY,2024-10-21,583.85,584.85,580.6001,583.63,36439010.0,386011.0,582.897586,588.15,0.007745


In [6]:
data_df.tail()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
173740,WM,2024-11-12,225.00,226.4281,223.81,224.78,2002438.0,35581.0,224.980260
173741,WM,2024-11-13,225.08,226.8400,224.24,225.68,1087208.0,25970.0,225.722744
173742,WM,2024-11-14,224.66,225.5500,221.57,221.72,1215866.0,28721.0,222.357046
173743,WM,2024-11-15,219.51,221.5250,217.25,217.90,1796543.0,35639.0,218.632828
173744,WM,2024-11-18,217.31,219.4700,217.08,218.31,986891.0,25718.0,218.259074


In [7]:
data_df.timestamp.nunique(),  baseline_df.timestamp.nunique()

(1755, 1735)

In [8]:
data_df_1 = data_df[data_df.timestamp.isin(baseline_df.timestamp.unique())]
data_df_1.tail()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
173720,WM,2024-10-15,214.54,216.0100,213.430,213.79,1180409.0,29135.0,214.461648
173721,WM,2024-10-16,213.22,214.2024,212.515,213.14,718619.0,19140.0,213.193734
173722,WM,2024-10-17,214.13,214.4900,212.140,213.08,847090.0,20474.0,212.939363
173723,WM,2024-10-18,213.39,213.5000,212.000,212.75,816092.0,17581.0,212.948471
173724,WM,2024-10-21,212.56,212.9300,210.370,211.82,1033029.0,25100.0,211.721782


In [9]:
processed_df.tail()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,target_,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
171760,WM,2024-10-15,214.54,216.0100,213.430,213.79,1180409.0,29135.0,214.461648,224.78,...,0.731010,0.369566,0.361444,-9.074876,-7.096165,-1.978710,209.000931,0.135831,0.135739,196.600999
171761,WM,2024-10-16,213.22,214.2024,212.515,213.14,718619.0,19140.0,213.193734,225.68,...,0.783440,0.452341,0.331099,-11.619399,-8.000812,-3.618587,209.319498,-0.304037,-0.304500,195.699223
171762,WM,2024-10-17,214.13,214.4900,212.140,213.08,847090.0,20474.0,212.939363,221.72,...,0.813041,0.524481,0.288560,-12.726385,-8.945927,-3.780459,209.513961,-0.028151,-0.028154,195.615982
171763,WM,2024-10-18,213.39,213.5000,212.000,212.75,816092.0,17581.0,212.948471,217.90,...,0.814295,0.582444,0.231851,-13.766248,-9.909991,-3.856257,209.739171,-0.154871,-0.154991,195.158158
171764,WM,2024-10-21,212.56,212.9300,210.370,211.82,1033029.0,25100.0,211.721782,218.31,...,0.770774,0.620110,0.150664,-12.850104,-10.498014,-2.352091,210.035791,-0.437133,-0.438091,193.867925


In [10]:
baseline_df.shape, data_df.shape, processed_df.shape

((1735, 11), (173745, 9), (171765, 97))

In [11]:
with open("/home/naradaw/dev/Charles_Schwab/data/symbol_universe/snp_unique_100_2019", "rb") as fp:
    symbol_universe = pickle.load(fp)
    
symbol_universe = symbol_universe[:20]

In [12]:
print(symbol_universe)

['SWKS', 'ALLE', 'BDX', 'CMI', 'APH', 'PNC', 'GWW', 'GLW', 'NRG', 'JKHY', 'LRCX', 'KHC', 'OMC', 'KMX', 'MRO', 'GE', 'NVDA', 'NOC', 'KO', 'CE']


In [13]:
with open(data_sqs_path, 'rb') as f:
    data = pickle.load(f)

features = np.array([data[symbol]['features'] for symbol in symbol_universe])
returns = np.array([data[symbol]['returns'] for symbol in symbol_universe])


In [14]:
baseline_returns = data['baseline_return']
baseline_returns

array([-0.05680392, -0.04213348, -0.04583587, ...,  0.01888898,
        0.0019843 ,  0.00774463])

In [15]:
features.shape

(20, 1676, 60, 87)

In [16]:
len(features.transpose(1,0,2,3)) , len(data['baseline_return'])

(1676, 1676)

In [17]:
data.keys()

dict_keys(['A', 'AJG', 'ALB', 'ALL', 'ALLE', 'ANET', 'APA', 'APD', 'APH', 'AWK', 'AXP', 'BBY', 'BDX', 'BEN', 'BIIB', 'CAG', 'CAH', 'CE', 'CF', 'CHD', 'CHTR', 'CINF', 'CMI', 'CPB', 'CSCO', 'CTAS', 'DAL', 'DFS', 'DG', 'DHI', 'EBAY', 'ED', 'EIX', 'EL', 'EMR', 'ES', 'ETN', 'EXPE', 'FMC', 'FRT', 'GE', 'GL', 'GLW', 'GWW', 'HCA', 'HRL', 'HUM', 'IFF', 'INTC', 'IPG', 'IRM', 'JCI', 'JKHY', 'JNJ', 'JPM', 'KHC', 'KMB', 'KMX', 'KO', 'KR', 'LRCX', 'LYB', 'MA', 'MCK', 'MDLZ', 'MLM', 'MMM', 'MRO', 'MU', 'NCLH', 'NOC', 'NRG', 'NVDA', 'NWS', 'OMC', 'ORLY', 'PNC', 'PNR', 'PYPL', 'QRVO', 'REG', 'REGN', 'RF', 'RMD', 'ROP', 'ROST', 'RSG', 'SBUX', 'SHW', 'SWKS', 'TAP', 'TSN', 'TTWO', 'UAL', 'VRSN', 'VTR', 'VZ', 'WEC', 'WM', 'baseline_return'])

In [18]:
del data

In [19]:
features.shape, returns.shape

((20, 1676, 60, 87), (20, 1676))

In [20]:
features = features.transpose(1,2,0,3)
returns = returns.transpose(1,0)

In [21]:
features.shape, returns.shape

((1676, 60, 20, 87), (1676, 20))

In [22]:
returns[0].shape

(20,)

In [23]:
returns[: , 1]

array([-0.03486031, -0.0350574 , -0.03179551, ..., -0.07318993,
       -0.08868023, -0.08795474])

In [24]:
baseline_returns

array([-0.05680392, -0.04213348, -0.04583587, ...,  0.01888898,
        0.0019843 ,  0.00774463])